## Overview
This notebook trains the model used by the `audio-denoiser` Python package: https://github.com/jose-solorzano/audio-denoiser



## Huggingface Key
If `save_model` is True in the training method call, we need a Huggingface key/token, which needs to be set as HF_KEY in the Secrets Add-on of this Kaggle notebook.

In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_KEY")

## Configure system path
Script `tools/upload_code.py` from the Github repo uploads training source code to a GCS bucket (set up in `config.py`), which is then pulled by
Kaggle dataset `js-audio-denoiser`. The code folder is added to `sys.path`.

In [2]:
import sys 
import os
import sklearn
import torch
import torchaudio

code_path = '/kaggle/input/js-audio-denoiser'
ts_path = os.path.join(code_path, '.timestamp.txt')
with open(ts_path) as f:
    contents = f.read()
    print(f'Timestamp: {contents}')
sys.path.insert(1, code_path)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Timestamp: 2023-10-21 11:34:06.744689


## Configure the right audio backend
The`soundfile` backend should be supported by both Windows and Linux. The `sox_io` backend, used by default in Linux, appears to load data with a different distribution than `soundfile`.

In [3]:
torchaudio.set_audio_backend("soundfile")

## Check CUDA status

In [4]:
import gc

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    free_mem, total_mem = torch.cuda.mem_get_info()
    print(f'CUDA free={free_mem/1e9:.4g} Gb, total={total_mem/1e9:.4g} Gb')
else:
    print('CUDA not available!')

CUDA free=16.43 Gb, total=17.07 Gb


## Training

In [5]:
from tools import train_model

# Test training
train_model.main_impl(num_epochs=2, save_model=False, hf_token=hf_token)

## Actual training
# train_model.main_impl(num_epochs=2500, save_model=True, hf_token=hf_token)

Device: cuda:0
Finding audio file paths...
Found 227118 audio files.
Reading ambient noise files...
Training...
Training scaler with 100 audio files...
Building dataset...
Training dataset size: 2000
Epoch [1/2] Loss: 1.373
Epoch [2/2] Loss: 1.031
Evaluating...
Validation epoch 1...
MSE: 0.9838 - Relative variance: 0.4755
Validation epoch 2...
MSE: 0.967 - Relative variance: 0.4747
Validation epoch 3...
MSE: 0.9768 - Relative variance: 0.4823
Experiment 115603 did not save a model.
